In [1]:
from datasets import load_dataset

dataset = load_dataset("Yelp/yelp_review_full", cache_dir="./data/hf/datasets")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})


In [2]:
print(dataset["train"][0])

{'label': 4, 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}


In [3]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"])

,label,text
0,1 star,"This is by far the skankiest, worst, most dubious Walmart in the Valley. I'd rather go without than step foot into this store. It's always more crowded than usual (Walmarts suck! Plan to stand in line FOREVER!) but this one takes the cake. From the throngs of people too fat to walk motoring around on their scooters (Hey, I'm calling it like I see it, hate me if you want), people clogging up the already enflamed lines by price matching 10 different grocery store ads, to paying for conveyor belts brimming with flaming hot Cheetos, candy, Powerade and other various almost non-food items with food stamps, to gang bangers and cursing teenagers in line giving me the stink eye (why does no one care to curb their profanity in front of kids anymore?) to employees who can't direct me to the simplest things, I HATE THIS STORE. Please, dear God, corporate moguls hear my plea, open a Super Target in Laveen."
1,5 stars,"This is basically for Shaina who with unbelievable skill and dexterity, put on my screen protector for my iPhone 6 Plus. She was great and I was definitely happy I stopped in. And overall really friendly and helpful staff."
2,1 star,This place is still open because the snowbirds don't know what real Mexican food is all aabout and there is no where else in AJ to eat. If you have a car there is no reason to eat here at all.
3,1 star,"Placed an order for delivery with this store today, and was called later and told they couldn't deliver it because \""he was the only one there today\"" so I would need to pick it up, but I would get a discount. As I had already submitted my credit card information when I placed the order, and didn't want to be charged for an order I didn't get, I went to pick it up. \n\nWhen I arrived, there were at least 2 people that I could see working. As I had put the tip amount in when I placed the order online, I told him that I would need that amount removed, as I wasn't going to tip them for NOT delivering. Instead, he ran it through with the amount on. I noticed this before I signed, and told him I would not sign as it was incorrect. After quite a bit of trouble he finally reversed the charges and charged me the correct amount.\n\nFor the hassle of picking up, when I had specifically chosen the restaurant due to the delivery option, I should have been given a discount or a free item. Neither was given to me as he said he \""couldn't\"" once I arrived, nor was an apology for them not being able to offer the service that they clearly offer.\n\nDue to this, I will not be ordering from here again. There are many, many other restaurants in the Madison area, and I will be patronizing any of them instead of a restaurant that does not provide the service they advertise."
4,2 star,"I have a love-hate relationship with this place... \n\nI love it because it's extremely close to me (so, it's convenient when I'm craving a beer) and the atmosphere on Sundays is outstanding. I'm a giant Steeler fan, but you'll probably find fans from all 32 teams in Buffalo blues, which gives it a fun and exciting feel during NFL Sundays. \n\nHowever, I hate it because I believe the alcohol and food is overpriced. The food wouldn't rub me the wrong way so much if I thought it was worth the price, but I don't. Not even a little bit. The only thing I have found to be delicious there were the wings; however, you can go to almost any other bar to get delicious and cheaper wings. So, it's not really much of a win at all. Also, I want to have a handful of beers during the games, but I don't want to empty my pockets doing it. So, all in all, I recommend going once and awhile (mainly on Sundays) for a beer or two, but other than that it's not worth much of a visit."
5,3 stars,"I don't imagine this being a restaurant that you would go to unless you were a guest at the El Cortez hotel, or happened to be hanging around the hotel casino, got hungry, and decided you wanted to eat diner food. In short, this place is a downsca

In [6]:
import os

os.environ['HF_HOME'] = './data/hf'
os.environ['HF_HUB_CACHE'] = './data/hf/hub'

In [7]:
from transformers import AutoTokenizer

model_name = 'google-bert/bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [8]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,2 star,"My boyfriend's dad gave us the 2011 Entertainment Guide book for Christmas and, slowly, we have been exploring new-to-us-restaurants with our coupons. His dad also has the same book and used his coupon for Anntony's yesterday for lunch. His dad is not one to call us and rave about a place, but he said we needed to check it out. We checked out the menu that was advertised in the book to see if the food matched our tastes. I was excited to see they offered Caribbean Crab Cakes so we headed out to Anntony's for dinner.\n\nWhen we walked in at 6:45, there was not a single person in the restaurant. We sat down and were handed menu's that differed from the menu in the entertainment book. I checked their menu on-line after dinner and found it to be correct with the menu at the restaurant though. The menu consisted of all chicken and some pork options. I was really disappointed to find that no crab-cake listed. In addition, their menu did not supply a description of the entrees, but it did for their sides. I found the menu very underwhelming. \n\nI ordered the chicken wing dinner that was tossed in a special Caribbean sauce with their black beans and mac and cheese. My boyfriend ordered the chicken breast with collard greens and rice. The wings were okay, some were dry and some were not. He said his chicken tasted like it was leftover from lunch and was dry as well. The beans, mac and cheese, and rice were very good though. \n\nTheir slogan is \""Don't Worry About Nothin' Mon!\"" I WISH! I was very worried while I was dining for the following reasons: \n-They had two TVs on that showed depressing world events (Not sure why a restaurant has TVs in it, it's not a bar). \n-The server, although very pleasant and sweet, did not have her hair tied back and wore a strapless dress with flip flops. We asked for hot sauce and she brought out a bottle of sauce with out a lid on it. Where was the Health Department when we needed them? \n-Why were we the only people in the restaurant??? \n\n\nI can only assume that my boyfriends dad liked the place so much was due to the amount of food you get with the low priced meals and that lunch was a bit fresher than dinner. \n\nI would not recommend this place for dinner. Maybe lunch is better, but the food safety and health issues are a bit concerning.","[101, 1422, 6508, 112, 188, 4153, 1522, 1366, 1103, 1349, 4549, 7500, 1520, 1111, 3394, 1105, 117, 2494, 117, 1195, 1138, 1151, 12138, 1207, 118, 1106, 118, 1366, 118, 7724, 1114, 1412, 8707, 4199, 119, 1230, 4153, 1145, 1144, 1103, 1269, 1520, 1105, 1215, 1117, 8707, 1320, 1111, 5083, 25843, 112, 188, 8128, 1111, 5953, 119, 1230, 4153, 1110, 1136, 1141, 1106, 1840, 1366, 1105, 187, 8308, 1164, 170, 1282, 117, 1133, 1119, 1163, 1195, 1834, 1106, 4031, 1122, 1149, 119, 1284, 6488, 1149, 1103, 13171, 1115, 1108, 18428, 1107, 1103, 1520, 1106, 1267, 1191, 1103, 2094, 10260, 1412, 18689, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from transformers import TrainingArguments

model_dir = "./data/hf/models/bert-base-cased-finetune-yelp"
training_args = TrainingArguments(
    output_dir=model_dir, per_device_train_batch_size=16, num_train_epochs=5, logging_steps=100
)
print(training_args)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`